# Data exploration notebook
uitleg hoe tabellen samenhangen:

1. Hoofdconcepten en Entiteiten

- Archer: Bevat gegevens over de boogschutters zoals naam, geslacht, categorie, en vereniging.
- Bow: Informatie over bogen zoals type, handgreep, en status (actief of gearchiveerd).
- Arrowset en Arrow: Bevat gegevens over pijlen en pijlenstellen, inclusief specificaties zoals gewicht en diameter.
- Bowsetup: Gedetailleerde configuratie van een boog, inclusief gerelateerde boog en richtmiddelen (via referenties naar Bow en Sight).

2. Wedstrijden en Schietprestaties

- Match: Bevat gegevens over wedstrijden, inclusief locatie, datum, en specifieke instellingen zoals maximale series en schoten.
- Serie: Een reeks schoten binnen een wedstrijd, met locatie en geschatte afstanden.
- Shot: Gedetailleerde gegevens van individuele schoten, inclusief trefpunt (x, y), waarde, tijd, en gerelateerde hartslagdata. Elke Shot verwijst naar een Serie en kan een specifieke pijl (via Arrow) betreffen.
- Motion: Data over bewegingen tijdens een schot, met een verwijzing naar een Shot.

3. Uitrusting en Configuratie

- Sight: Bevat gegevens over vizieren en hun instellingen.
- SightSetting: Specifieke instellingen voor vizieren, gekoppeld aan een boogsetup en pijlenstel.

4. Rondes en Analyses

- Round en RoundEntry: Gegevens over rondes binnen wedstrijden, met koppelingen naar specifieke matches.
- Filter en FilterSet: Hulpmiddelen voor het filteren en analyseren van data, zoals schoten, wedstrijden of uitrusting.

5. Vragenlijsten en Psychologische Data

- Questionaire en Answer: Voor het bijhouden van antwoorden op vragenlijsten. Elke antwoord is gekoppeld aan een vragenlijst en een vraag.
- Kuiperslist: Mogelijk een specifieke psychologische schaal met ja/nee-vragen (1 of 0).
- Question: Bevat vragen die gebruikt worden in vragenlijsten, met aanvullende metadata zoals kleurinstellingen.

6. Tags en Metadata
- Tag en MatchTag: Tags voor het categoriseren van wedstrijden.
- MatchPhoto: Foto's gekoppeld aan wedstrijden.
- android_metadata: Metadata voor mobiele toepassingen (mogelijk voor internationalisering of versiebeheer).

7. Hartslag en Stressgegevens
- Heartrate: Bevat hartslaggegevens zoals gemiddelde BPM, stressindex en variabiliteit (SDNN, RMSSD, PNN50).


## Relaties Samengevat

- Boogschutter naar Uitrusting: Een boogschutter (Archer) gebruikt een bepaalde boog (Bow), pijlenstel (Arrowset), en boogconfiguratie (Bowsetup).
- Uitrusting naar Wedstrijden: Een boogconfiguratie (Bowsetup) en pijlenstel (Arrowset) worden gebruikt in wedstrijden (Match).
- Wedstrijden naar Data: Een wedstrijd (Match) bevat series (Serie) die uit individuele schoten (Shot) bestaan.
- Data-analyse: Filters (Filter, FilterSet) kunnen worden toegepast op schoten, wedstrijden of andere gegevens.
- Psychologische Data: Psychologische evaluaties worden uitgevoerd via vragenlijsten (Questionaire, Kuiperslist) met antwoorden in Answer.



In [2]:
import sqlite3
import pandas as pd
import os
pd.set_option('display.max_columns', None)

## match table

In [22]:
conn = sqlite3.connect("./dbToAdd.db")
df_match = pd.read_sql_query("SELECT * FROM match", conn)

# andere manier, probleem hier is dat het de kolomnamen niet meeneemt:
""" c = conn.cursor()
c.execute('SELECT * FROM match')
data = c.fetchall()
conn.close()
# Convert the data to a pandas DataFrame
df = pd.DataFrame(data) """

df_match.head()


,_id,_time_c,_time_u,location,date,type,competition,face,distance,distance_units,...,meteo_winddir,notes,unrecorded_volume,unrecorded_score,time_prep,time_shoot,time_warn,forced_end,archived,_aid
0,1,1649573710,1649573710,lowlands almere,1649548800,1,1,10,70.0,0,...,NaN,None,None,None,None,None,None,None,0,8taWcn7KldKx
1,2,1649581541,1649581541,lowlands almere score 2,1649548800,1,1,10,70.0,0,...,NaN,None,None,None,None,None,None,None,0,8taWcn7KldKx
2,3,1650011153,1650040560,training ksm,1649980800,1,2,10,70.0,0,...,NaN,pees oude fast flight geel,None,None,None,None,None,None,0,8taWcn7KldKx
3,4,1650040466,1650040466,training ksm,1649980800,1,2,10,70.0,0,...,NaN,pees spectra 18 draads,None,None,None,None,None,None,0,8taWcn7KldKx
4,5,1650043914,1650043914,training ksm,1649980800,1,2,10,70.0,0,...,NaN,tiller 3mm,None,None,None,None,None,None,0,8taWcn7KldKx


In [23]:
# drop distance_units, sightadviceaccept_shot_id, time_prep, time_shoot, time_warn, forced_end
df_match = df_match.drop(columns=["distance_units", "sightadviceaccept_shot_id", "time_prep", "time_shoot", "time_warn", "forced_end"])
df_match.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,1649573710,1649573710,lowlands almere,1649548800,1,1,10,70.0,6,6,1,1,NaN,2.0,NaN,None,None,None,0,8taWcn7KldKx
1,2,1649581541,1649581541,lowlands almere score 2,1649548800,1,1,10,70.0,6,6,1,1,2.0,3.0,NaN,None,None,None,0,8taWcn7KldKx
2,3,1650011153,1650040560,training ksm,1649980800,1,2,10,70.0,9,4,1,1,3.0,NaN,NaN,pees oude fast flight geel,None,None,0,8taWcn7KldKx
3,4,1650040466,1650040466,training ksm,1649980800,1,2,10,70.0,9,4,1,1,2.0,NaN,NaN,pees spectra 18 draads,None,None,0,8taWcn7KldKx
4,5,1650043914,1650043914,training ksm,1649980800,1,2,10,70.0,6,6,1,1,NaN,NaN,NaN,tiller 3mm,None,None,0,8taWcn7KldKx


In [25]:
df_match.isnull().sum()

_id                    0
_time_c                0
_time_u                0
location               0
date                   0
type                   0
competition            0
face                   0
distance               0
maxshotsinserie        0
maxseries              0
bowsetup_id            0
arrowset_id            0
meteo_weather        215
meteo_wind           284
meteo_winddir        290
notes                244
unrecorded_volume     69
unrecorded_score     295
archived               0
_aid                   0
dtype: int64

In [ ]:
# _time_c, _time_u, date parsen vanuit unix epoch
df_match['_time_c'] = pd.to_datetime(df_match['_time_c'], unit='s')
df_match['_time_u'] = pd.to_datetime(df_match['_time_u'], unit='s')
df_match['date'] = pd.to_datetime(df_match['date'], unit='s') 

# distance parsen naar int
#df_match['distance'] = df_match['distance'].astype(int) #use floats instead of ints to allow for NaN values

df_match.head()

In [29]:
# type, competition, face, meteo_weather, meteo_wind, decoden 
# Mapping dictionaries
type_mapping = {
    1: 'WA Outdoor',
    3: 'WA Indoor',
    0: 'Custom',
    12: 'FITA Outdoor'
}
competition_mapping = {
    0: 'training',
    1: 'competition',
    2: 'tuning'
}
face_mapping = {
    0: 'blank bale',
    10: '122cm',
    2: ''
}
metheo_weather_mapping = {
    0: 'sunny',
    1: 'cloudy with sun',
    2: 'cloudy',
    3: 'light rain',
    4: 'heavy rain',
    5: 'indoor',
}
meteo_wind_mapping = {
    0: '0bft',
    1: '1bft',
    2: '2bft',
    3: '3bft',
    4: '4bft',
    5: '5bft',
    6: '6bft',
    7: '7bft',
    8: '8bft',
    9: '9bft'
}
# Replace numerical values with string labels
df_match['type'] = df_match['type'].replace(type_mapping)
df_match['competition'] = df_match['competition'].replace(competition_mapping)
df_match['face'] = df_match['face'].replace(face_mapping)
df_match['meteo_weather'] = df_match['meteo_weather'].replace(metheo_weather_mapping)
df_match['meteo_wind'] = df_match['meteo_wind'].replace(meteo_wind_mapping)
df_match.head()

,_id,_time_c,_time_u,location,date,type,competition,face,distance,maxshotsinserie,maxseries,bowsetup_id,arrowset_id,meteo_weather,meteo_wind,meteo_winddir,notes,unrecorded_volume,unrecorded_score,archived,_aid
0,1,2022-04-10 06:55:10,2022-04-10 06:55:10,lowlands almere,2022-04-10,WA Outdoor,competition,122cm,70.0,6,6,1,1,NaN,2bft,NaN,None,None,None,0,8taWcn7KldKx
1,2,2022-04-10 09:05:41,2022-04-10 09:05:41,lowlands almere score 2,2022-04-10,WA Outdoor,competition,122cm,70.0,6,6,1,1,cloudy,3bft,NaN,None,None,None,0,8taWcn7KldKx
2,3,2022-04-15 08:25:53,2022-04-15 16:36:00,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,9,4,1,1,light rain,NaN,NaN,pees oude fast flight geel,None,None,0,8taWcn7KldKx
3,4,2022-04-15 16:34:26,2022-04-15 16:34:26,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,9,4,1,1,cloudy,NaN,NaN,pees spectra 18 draads,None,None,0,8taWcn7KldKx
4,5,2022-04-15 17:31:54,2022-04-15 17:31:54,training ksm,2022-04-15,WA Outdoor,tuning,122cm,70.0,6,6,1,1,NaN,NaN,NaN,tiller 3mm,None,None,0,8taWcn7KldKx


## round table

In [4]:
conn = sqlite3.connect("./dbToAdd.db")
df_round = pd.read_sql_query("SELECT * FROM round", conn)


df_round.head()

,_id,_time_c,_time_u,name,location,date,type,competition,notes,archived,_aid


## roundentry table

In [5]:
conn = sqlite3.connect("./dbToAdd.db")
df_roundentry = pd.read_sql_query("SELECT * FROM roundentry", conn)


df_roundentry.head()

,_id,_time_c,_time_u,n,round_id,match_id,_aid


## serie table

In [6]:
conn = sqlite3.connect("./dbToAdd.db")
df_serie = pd.read_sql_query("SELECT * FROM serie", conn)


df_serie.head()

,_id,_time_c,_time_u,estimated_face,estimated_distance,estimated_distance_units,face,distance,distance_units,match_id,n,lattitude,longitude,altitude,marked,_aid
0,1,1649576991,1649576991,None,None,None,10,70.0,0,1,1,None,None,None,None,8taWcn7KldKx
1,2,1649577078,1649577078,None,None,None,10,70.0,0,1,2,None,None,None,None,8taWcn7KldKx
2,3,1649577921,1649577921,None,None,None,10,70.0,0,1,3,None,None,None,None,8taWcn7KldKx
3,4,1649578801,1649578801,None,None,None,10,70.0,0,1,4,None,None,None,None,8taWcn7KldKx
4,5,1649579721,1649579721,None,None,None,10,70.0,0,1,5,None,None,None,None,8taWcn7KldKx


## shot table

In [7]:
conn = sqlite3.connect("./dbToAdd.db")
df_shot = pd.read_sql_query("SELECT * FROM shot", conn)


df_shot.head()

,_id,_time_c,_time_u,reldiam,x,y,notation,value,isa,timing,match_id,serie_id,arrow_id,n,r,theta,boltrotation,heartrate,hrvar_rmssd,hrvar_pnn50,hrvar_sdnn,stress_index,speed,bowcant,_aid
0,1,1649577006,1649577006,0.078571,-1.949014,-1.960381,7,7,None,None,1,1,None,1,2.764372,-2.353287,None,None,None,None,None,None,None,None,8taWcn7KldKx
1,2,1649577029,1649577029,0.078571,-1.536498,1.169707,8,8,None,None,1,1,None,2,1.931073,2.490911,None,None,None,None,None,None,None,None,8taWcn7KldKx
2,3,1649577040,1649577040,0.078571,-1.549506,0.152702,9,9,None,None,1,1,None,3,1.557012,3.043361,None,None,None,None,None,None,None,None,8taWcn7KldKx
3,4,1649577049,1649577049,0.078571,0.916881,1.562156,8,8,None,None,1,1,None,4,1.811353,1.040040,None,None,None,None,None,None,None,None,8taWcn7KldKx
4,5,1649577061,1649577061,0.078571,0.177505,0.144166,X,10,None,None,1,1,None,5,0.228675,0.682122,None,None,None,None,None,None,None,None,8taWcn7KldKx


## matchtag table

In [8]:
conn = sqlite3.connect("./dbToAdd.db")
df_matchtag = pd.read_sql_query("SELECT * FROM matchtag", conn)


df_matchtag.head()

,_id,_time_c,_time_u,match_id,tag_id,_aid
0,1,1670580187,1670580187,264,1,8taWcn7KldKx


## tag table

In [9]:
conn = sqlite3.connect("./dbToAdd.db")
df_tag = pd.read_sql_query("SELECT * FROM tag", conn)


df_tag.head()

,_id,_time_c,_time_u,tag,_aid
0,1,1670580187,1670580187,nokpunt,8taWcn7KldKx


## other tables

In [ ]:
#TODO: pipeline van andere tabellen maken

In [ ]:
#TODO: tabellen joinen en samenvoegen